In [1]:
################################# BLOCK 1 : 공통 ################################# 

#필요한 모듈과 라이브러리를 로딩
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd    
import time, os, math, random

import pymysql
from selenium.webdriver.common.action_chains import ActionChains


# 사용자에게 크롤링 방식 묻기
print("=" *80)
print("         키워드 검색 쿠팡 크롤러입니다.")
print("=" *80)
print('\n')

#검색할 키워드와 크롤링할 건수 입력받기
keyword = str(input("1.크롤링할 키워드를 지정해주세요 : "))
cnt = int(input('2.크롤링 할 건수는 몇건입니까?: '))

#직접 검색 방식으로 크롤링할 경우에는 한 페이지당 최대 게시물이 72개.
#동시에 72개씩 볼때 가장 많은 게시물을 크롤링 가능
page_cnt = math.ceil(cnt/72)
f_dir = input("3.파일을 저장할 폴더명을 지정해주세요(기본경로:c:\\temp\\mysql2\\):")
if f_dir == '' :
    f_dir = "c:\\temp\\mysql2\\"    

print("=" *80)
print("데이터 크롤링을 시작합니다.")
print("=" *80)

# 작업 시간과 고유 dir 등 생성
n = time.localtime()
s = '%04d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday)
s1 = '%02d-%02d' % (n.tm_hour, n.tm_min)
s_time = time.time( )

# 웹사이트 접속 후 해당 메뉴로 이동
chrome_path = "c:/temp/python/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)
query_url= ('https://www.coupang.com/np/search?component=&q={}&channel=user'. format(keyword))
driver.get(query_url)
driver.implicitly_wait(5)

# 72개씩 보기로 변경 - 액션 체인으로 마우스 가져다대고 클릭
time.sleep(random.randint(1,3))
from selenium.webdriver.common.action_chains import ActionChains
action = ActionChains(driver)
selectbox = driver.find_element_by_class_name('selectbox-options')
action.move_to_element(selectbox).perform()
time.sleep(0.5)
driver.find_element_by_xpath('''//*[@id="searchSortingList"]/ul/li[4]/label''').click()
driver.implicitly_wait(3)
time.sleep(random.randint(1,3))

# 웹 파싱을 위한 사전 준비
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 크롤링 데이터가 저장될 폴더를 날짜/시간/파일이름 순으로 저장
try: os.makedirs(f_dir+s+'\\'+s1+'\\'+keyword)
except : pass
os.chdir(f_dir+s+'\\'+s1+'\\'+keyword)

# 이미지/txt/csv 이름 지정
ff_dir=f_dir+s+'\\'+s1+'\\'+keyword
ff_name=f_dir+s+'\\'+s1+'\\'+keyword+'\\'+s+keyword+'.txt'
fc_name=f_dir+s+'\\'+s1+'\\'+keyword+'\\'+s+keyword+'.csv'

# 제품 이미지 저장용 폴더 생성
img_dir = ff_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

# 내용을 수집
print("\n")
print("========== 곧 수집된 결과를 출력합니다 ========== ")
print("\n")


# 크롤링 데이터가 들어갈 리스트 생성
ranking2=[]        #제품의 판매순위 저장
title2=[]          #제품 정보 저장
price2=[]          #현재 판매가 저장
sat_count2=[]      #상품평 수 저장
stars2 = []        #상품평점 저장
keyword2 = []      #크롤링 검색어

file_no = 0   # 이미지 파일 저장할 때 번호
count = 1     # 총 게시물 건수 카운트 변수


################################# BLOCK 2 : 크롤러 ################################# 

#각 페이지별 소스를 파싱해서 게시글 단위로 크롤링
for x in range(1,page_cnt + 1) :
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    item_result = soup.find('div','search-content search-content-with-feedback')
    item_result2 = item_result.find_all('li')

    for li in item_result2:

        if cnt < count :
            break

        # 제품 이미지 다운로드
        import urllib.request
        import urllib
        
        try :
            photo = li.find('dt','image').find('img')['src']
        except AttributeError :
            continue

        file_no += 1
        full_photo = 'https:' + photo
        urllib.request.urlretrieve(full_photo,str(file_no)+'.jpg')
        time.sleep(0.5)

        #제품 내용 추출
        f = open(ff_name, 'a',encoding='UTF-8')
        f.write("-----------------------------------------------------"+"\n")
        print("-" *70)

        ranking = count
        print("1.판매순위:",ranking)
        f.write('1.판매순위:'+ str(ranking) + "\n")

        try :
            t = li.find('div',class_='name').get_text().replace("\n","")
        except AttributeError :
            title = '제품소개가 없습니다'
            print(title.replace("\n",""))
            f.write('2.제품소개:'+ title + "\n")
        else :
            title = t.replace("\n","").strip()
            print("2.제품소개:", title.replace("\n","").strip())                  
            f.write('2.제품소개:'+ title + "\n")

        try :
            price = li.find('strong','price-value').get_text()
        except :
            price = '0'
            print("3.판매가격:", price.replace("\n",""))
            f.write('3.판매가격:'+ price + "\n")
        else :
            print("3.판매가격:", price.replace("\n",""))
            f.write('3.판매가격:'+ price + "\n")

        try :
            sat_count_1 = li.find('span','rating-total-count').get_text()
            sat_count_2 = sat_count_1.replace("(","").replace(")","")
        except  :
            sat_count_2='0'
            print('4.상품평 수: ',sat_count_2)
            f.write('4.상품평 수:'+ sat_count_2 + "\n")
        else :
            print('4.상품평 수:',sat_count_2)
            f.write('4.상품평 수:'+ sat_count_2 + "\n")

        try :
            stars1 = li.find('em','rating').get_text()
        except  :
            stars1='0'
            print('5.상품평점: ',stars1)
            f.write('5.상품평점:'+ stars1 + "\n")
        else :
            print('5.상품평점:',stars1)
            f.write('5.상품평점:'+ stars1 + "\n")

        print("6.키워드:",keyword)
        f.write('6.키워드:'+ str(keyword) + "\n")


        print("-" *70)

        f.close( )             
        time.sleep(0.5)

        #추출한 데이터를 리스트화
        ranking2.append(ranking)
        title2.append(title)
        price2.append(price)
        sat_count2.append(sat_count_2)
        stars2.append(stars1)
        keyword2.append(str(keyword))

        count += 1

    # 페이지 번호를 넘기고, 다음 페이지 번호 클릭
    x += 1          
    try :
        driver.find_element_by_class_name("btn-page").find_element_by_link_text('%s' %x).click() # 다음 페이지번호 클릭
    except :
        pass

    time.sleep(2)  
    
################################# BLOCK 3 : 데이터 정리 ################################# 
               
# 크롤링한 데이터를 pandas를 이용해 DataFrame 형태로 저장             
coupang_df = pd.DataFrame()
coupang_df['판매순위']=ranking2
coupang_df['제품소개']=pd.Series(title2)
coupang_df['제품판매가']=pd.Series(price2)
coupang_df['상품평수']=pd.Series(sat_count2)
coupang_df['상품평점']=pd.Series(stars2)
coupang_df['키워드']=pd.Series(keyword2)

# csv 형태로 저장하기
coupang_df.to_csv(fc_name,encoding="utf-8-sig",index=False)

e_time = time.time( )
t_time = e_time - s_time

count -= 1

print("\n")
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("=" *80)

driver.close( )


################################# BLOCK 4 : mySQL ################################# 

# 크롤링 데이터를 저장할지 여부 묻기
# STEP1. 크롤링 데이터를 DB에 저장할지 여부 묻고, 보여주기
db_save = input('크롤링된 데이터를 DB에 저장하시겠습니까? (Y/N):')
print('입력값 : ', db_save)

# STEP2. 크롤링 데이터를 저장한다고 답한 경우
if db_save =='y' or db_save == 'Y' or db_save == 'ㅛ':
    print('크롤링 데이터를 저장합니다.')
    # 데이터를 저장할 DataBase 이름 입력받기
    db_name = str(input('''데이터를 저장할 DataBase명을 입력해주세요. (기본 db : test_db) :'''))
    if db_name == "" : 
        db_name = "test_db"
    
    # 테이블 이름 입력, 테이블 이름만 입력해도 상관없으나 데이터를 불러올 때는 db명.table명 으로 호출해야 한다.
    # ex) SELECT * FROM test_db.table
    table_name = str(input('''데이터를 저장할 table명을 입력해주세요. (기본 table : test_db.table) : '''))
    if table_name == "" :
        table_name = "test_db.table"

    # mySQL과 연결하기 위한 필수값 설정
    host_name = "localhost"
    username = "root"
    password = "oracle"
    database_name = "mysql"

    # mySQL과 연결
    con = pymysql.connect(host=host_name,port=3306,user=username, passwd=password, db=database_name,charset='utf8')
    cur = con.cursor()

    # STEP3. 데이터 베이스 생성 or 접속
    try :
        SQL_QUERY = """ CREATE DATABASE """ + db_name + """ ; """
        cur.execute(SQL_QUERY)
    except :
        db_choice = input("Database가 이미 존재합니다. 해당 DB에 접속하거나(Y), 중복 DB를 제거하고 새로 만듭니다(N) :")
        if db_choice == 'n' or db_choice == 'N' or db_choice == 'ㅜ': 
            SQL_QUERY = """ DROP DATABASE IF EXISTS """ + db_name + """ ; """      
            cur.execute(SQL_QUERY)

            SQL_QUERY = """ CREATE DATABASE """ + db_name + """ ; """
            cur.execute(SQL_QUERY)
        else : pass

    # STEP4. 해당 데이터베이스 접속, 사용
    SQL_QUERY=""" USE """ + db_name + """ ; """
    cur.execute(SQL_QUERY)

    # STEP5. 테이블 생성 or 이어쓰기
    try :
        SQL_QUERY = """ CREATE TABLE """ + table_name + """ (
        RANKING INT, 
        TITLE VARCHAR(256) ,
        PRICE VARCHAR(10) ,
        REVIEW INT ,
        STARS DECIMAL(2,1) ,
        KEYWORD VARCHAR(20) ,
        CRAWLED DATETIME DEFAULT CURRENT_TIMESTAMP
        ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin 
        """
        cur.execute(SQL_QUERY)

    except :
        print("같은 이름의 테이블이 이미 존재합니다.") 
        table_choice = input("테이블에 데이터를 추가하거나(Y), 테이블을 삭제하고 새로 만듭니다(N) :")

        if table_choice ==  'n' or table_choice == 'N' or table_choice == 'ㅜ' :

            SQL_QUERY=""" DROP TABLE IF EXISTS """ + table_name + """ ; """
            cur.execute(SQL_QUERY)

            SQL_QUERY = """ CREATE TABLE """ + table_name + """ (
            RANKING INT, 
            TITLE VARCHAR(256) ,
            PRICE VARCHAR(10) ,
            REVIEW INT ,
            STARS DECIMAL(2,1) ,
            KEYWORD VARCHAR(20) ,
            CRAWLED DATETIME DEFAULT CURRENT_TIMESTAMP
            ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin 
            """
            cur.execute(SQL_QUERY)

        else : pass

    con.commit()

    # STEP6. 테이블에 들어갈 값을 지정합니다.
    sql = ''' INSERT INTO ''' + table_name + ''' (RANKING, TITLE, PRICE, REVIEW, STARS, KEYWORD, CRAWLED)
            VALUES (%s,%s,%s,%s,%s,%s, CURDATE()) ''' 

    rows = [tuple(x) for x in coupang_df.values]
    cur.executemany(sql,rows)
    con.commit()

else : 
    print("크롤링 데이터를 저장하지 않습니다.")
    pass

print("=" *80)
print(" 작업이 완료되었습니다. ")
print('\n')
print(" 작업 폴더를 엽니다. ")
print("=" *80)

# 작업 완료 후 작업 폴더 열기
os.startfile(f_dir)

         키워드 검색 쿠팡 크롤러입니다.


1.크롤링할 키워드를 지정해주세요 : 커피
2.크롤링 할 건수는 몇건입니까?: 100
3.파일을 저장할 폴더명을 지정해주세요(기본경로:c:\temp\mysql2\):
데이터 크롤링을 시작합니다.


========== 곧 수집된 결과를 출력합니다 ========== 


----------------------------------------------------------------------
1.판매순위: 1
2.제품소개: 키토제닉라이프스타일 데일리키토 방탄 원두커피믹스, 14포, 10g
3.판매가격: 9,020
4.상품평 수: 407
5.상품평점: 5.0
6.키워드: 커피
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 2
2.제품소개: 곰곰 쓰지않고 부드러운 아메리카노 다크로스트, 1g, 100개입
3.판매가격: 10,400
4.상품평 수: 2659
5.상품평점: 4.0
6.키워드: 커피
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 3
2.제품소개: 곰곰 헤이즐넛아메리카노 원두커피믹스, 1g, 100개입
3.판매가격: 10,320
4.상품평 수: 841
5.상품평점: 4.5
6.키워드: 커피
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 4
2.제품

----------------------------------------------------------------------
1.판매순위: 33
2.제품소개: 프렌치카페 카페믹스, 10.9g, 330개
3.판매가격: 25,670
4.상품평 수: 19353
5.상품평점: 4.5
6.키워드: 커피
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 34
2.제품소개: 투썸플레이스 에이리스트 스틱 커피 다크 블렌드, 1.1g, 30개입
3.판매가격: 6,010
4.상품평 수: 1384
5.상품평점: 4.5
6.키워드: 커피
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 35
2.제품소개: 마케마케 설탕을 줄인 모카골드 라이트 커피믹스, 10.5g, 160개입
3.판매가격: 15,160
4.상품평 수: 257
5.상품평점: 4.0
6.키워드: 커피
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 36
2.제품소개: 프렌치카페 카페오레, 200ml, 10개
3.판매가격: 8,100
4.상품평 수: 3719
5.상품평점: 4.5
6.키워드: 커피
----------------------------------------------------------------------
-----------------------------

----------------------------------------------------------------------
1.판매순위: 67
2.제품소개: 프렌치카페 로스터리 모카, 250ml, 10개입
3.판매가격: 9,700
4.상품평 수: 1618
5.상품평점: 4.5
6.키워드: 커피
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 68
2.제품소개: 바바커피 롱블랙, 230ml, 10개
3.판매가격: 8,500
4.상품평 수: 44
5.상품평점: 4.5
6.키워드: 커피
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 69
2.제품소개: 뉴트리365 로얄 헤이즐넛향 고급 원두커피, 227g, 2개
3.판매가격: 11,950
4.상품평 수: 333
5.상품평점: 4.5
6.키워드: 커피
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 70
2.제품소개: 프렌치카페 로스터리 옐로우 버번 돌체 라떼, 250ml, 10개
3.판매가격: 8,810
4.상품평 수: 2876
5.상품평점: 4.5
6.키워드: 커피
----------------------------------------------------------------------
----------------------------------------



1.요청된 총 100 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 100 건입니다
2.총 소요시간은 124.3 초 입니다 
3.파일 저장 완료: txt 파일명 : c:\temp\mysql2\2020-09-25\14-33\커피\2020-09-25커피.txt 
4.파일 저장 완료: csv 파일명 : c:\temp\mysql2\2020-09-25\14-33\커피\2020-09-25커피.csv 
크롤링된 데이터를 DB에 저장하시겠습니까? (Y/N):y
입력값 :  y
크롤링 데이터를 저장합니다.
데이터를 저장할 DataBase명을 입력해주세요. (기본 db : test_db) :
데이터를 저장할 table명을 입력해주세요. (기본 table : test_db.table) : 
Database가 이미 존재합니다. 해당 DB에 접속하거나(Y), 중복 DB를 제거하고 새로 만듭니다(N) :y
 작업이 완료되었습니다. 


 작업 폴더를 엽니다. 


In [11]:
print('\n')
print("                             ==========         중략          ==========                       ")
print('\n')



                             ==========         중략          ==========                       


